# Toy example for Radix-based Benders

In [1]:
from gurobipy import *

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['svg.fonttype'] = 'none'
pd.set_option('display.max_colwidth', -1)
%matplotlib inline

from dynamicme.decomposition import Decomposer
from dynamicme.callback_gurobi import cb_benders
from dynamicme.optimize import Optimizer, StackOptimizer
from dynamicme.optimize import Constraint, Variable

from cobra.io import load_json_model
from cobra import Metabolite, Reaction
from six import iteritems

import numpy as np
import cobra

(<type 'exceptions.ImportError'>, ImportError('No module named cplex',), <traceback object at 0x7f46a2fb72d8>)


### Try optimizing using radix for one condition first

In [2]:
#----------------------------------------
# Starting from basal model
ijomc = load_json_model('/home/laurence/ME/models/BiGG_M/json/e_coli_core.json')
keff0 = 1./65/3600
crowding_bound = 0.0003
rxns_c = [r for r in ijomc.reactions if all([m.compartment=='c' for m in r.metabolites.keys()]) and 'BIOMASS' not in r.id]
crowding_dict = {rxn:keff0 for rxn in rxns_c}
#----------------------------------------

# Temporarily add crowding constraint for the duality gap constraint
crowding = Constraint('crowding')
crowding._bound = crowding_bound
crowding._constraint_sense = 'L'
for rxn,keff in iteritems(crowding_dict):
    rxn.add_metabolites({crowding:keff})

In [3]:
ijomc.optimize()

<Solution 0.67 at 0x7f46e552b910>

In [4]:
opt = Optimizer(ijomc)
gap = opt.add_duality_gap_constraint(INF=1e6)

In [5]:
gap.optimize()

<Solution 1.34 at 0x7f46e552b890>

In [6]:
import numpy as np

radix = 4.
powers = np.arange(-4,5)
print(powers)
digits_per_power = radix
pwr_max = max(powers)
digits = np.linspace(0, pwr_max-1, digits_per_power)
print(digits)

# Discretize crowding coefficients into radix
crowding_p = gap.metabolites.crowding

var_cons_dict = {}
for rxn_p in crowding_p.reactions:
    # Get the coefficient in the dual
    var_d = gap.reactions.wa_crowding
    cons_ds = [m for m in var_d.metabolites.keys() if rxn_p.id in m.id]
    a0 = rxn_p.metabolites[crowding_p]    
    var_cons_dict[rxn_p.id] = [(rxn_p, crowding_p, a0)] + [(var_d, cons_d, a0) for cons_d in cons_ds]

[-4 -3 -2 -1  0  1  2  3  4]
[0. 1. 2. 3.]


/usr/lib/python2.7/site-packages/ipykernel/__main__.py:8 DeprecationWarning: object of type <type 'float'> cannot be safely interpreted as an integer.


In [7]:
opt.to_radix(gap, var_cons_dict, radix, powers, digits_per_power)

Exception: Reactions already in the model: z_wa_crowding_G6PDH2r00

In [8]:
var_cons_dict['G6PDH2r']

[(<Variable G6PDH2r at 0x7f4642d2e790>,
  <Constraint crowding at 0x7f4642dd1b10>,
  4.273504273504274e-06),
 (<Variable wa_crowding at 0x7f4642d3ab10>,
  <Constraint G6PDH2r at 0x7f4642d00490>,
  4.273504273504274e-06)]

In [ ]:
for rxn in gap.reactions:
    rxn.objective_coefficient = 0.
# for rxn in gap.reactions.query('binary'):
#     rxn.objective_coefficient = 1.

In [ ]:
N_CONDS = 1

df_meas = pd.read_csv('/home/laurence/ME/data/dynamicME/beg/growth_meas.csv')
ex_rxns = [r.id for r in ijomc.reactions.query('EX_')]
df_meas = df_meas[ df_meas.ex_rxn.isin(ex_rxns)]
conds = df_meas.substrate.unique()
df_conds = pd.DataFrame([{'cond':r['substrate'], 'rxn':ex_rxn, 'lb':-10 if r['ex_rxn']==ex_rxn else 0, 'ub':1000., 'obj':0.} for i,r in df_meas.iterrows() for ex_rxn in ex_rxns])
df_conds = df_conds[ df_conds.cond.isin(conds[0:N_CONDS])]

In [ ]:
mu_meas = 0.74

F_TOL = 0.15

gap.reactions.BIOMASS_Ecoli_core_w_GAM.lower_bound = mu_meas*(1-F_TOL)
gap.reactions.BIOMASS_Ecoli_core_w_GAM.upper_bound = mu_meas*(1+F_TOL)

from cobra.solvers import gurobi_solver
from gurobipy import *

milp = gurobi_solver.create_problem(gap)
milp.ModelSense = GRB.MINIMIZE

In [ ]:
milp.Params.IntFeasTol = 1e-9
milp.Params.OutputFlag = 1
milp.optimize()

In [ ]:
#gap.optimize('minimize')
rxn_mu =gap.reactions.BIOMASS_Ecoli_core_w_GAM 
#muopt = rxn_mu
sol = gurobi_solver.format_solution(milp, gap)
muopt = sol.x_dict[rxn_mu.id]
print('%g <= %g <= %g' % (rxn_mu.lower_bound, muopt, rxn_mu.upper_bound))

In [ ]:
yopt = [sol.x_dict[rxn.id] for rxn in gap.reactions.query('binary_')]
sum(yopt)